# State Conservation Status Species Lists - ACT

This notebook downloads the ACT lists from the ACT Government Open Data Portal and readies them for ingestion into the ALA Lists tool. It will save original lists to the source-data/ACT directory, process the lists and save them to current-lists. The lists can be loaded into the lists-test environment then reviewed by following the instructions below.

## Lists in the ALA Species List tool
Conservation list: [dr649](https://lists.ala.org.au/speciesListItem/list/dr649) ([dr649 in test](https://lists-test.ala.org.au/speciesListItem/list/dr649))
Sensitive list: [dr2627](https://lists.ala.org.au/speciesListItem/list/dr2627) ([dr2627 in test](https://lists.ala.org.au/speciesListItem/list/dr2627))

## Sources

Source:
Metadata: https://www.data.act.gov.au/Environment/ACT-Nature-Conservation-Threatened-Native-Species-/9ikf-qahj
Data: https://www.data.act.gov.au/resource/9ikf-qahj.json

***Metadata summary***
ACT Nature Conservation Threatened Native Species List. Species list adheres to the Darwin Core Standard. A species is threatened if it is likely to become extinct in the foreseeable future. The Nature Conservation Act 2014 (ACT) establishes a formal process for the identification and protection of threatened species and ecological communities. Ngunnawal species names created and endorsed by the Winnagaay Ngunnawal Language Group.

***URL***
https://www.data.act.gov.au/Environment/ACT-Nature-Conservation-Threatened-Native-Species-/9ikf-qahj
https://dev.socrata.com/foundry/www.data.act.gov.au/9ikf-qahj/



In [3]:
# set up
import pandas as pd
import json
import datetime
import urllib.request
import certifi
import ssl
import os
import os.path as path

# top level directory
projectDir = path.abspath(path.join(os.getcwd(),"../../..")) + "/"
sourceDir = projectDir + "source-data/ACT/"
statusDir = projectDir + "source-data/status-codes/"
cdataDir = projectDir + "current-lists/conservation-lists/"

monthStr = datetime.datetime.now().strftime('%Y%m')

import sys
import os
sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
import list_functions as lf

# Conservation List

In [12]:
# %%script echo skipping # comment this line to download the files and save locally
# download and save the conservation list locally
with urllib.request.urlopen("https://www.data.act.gov.au/resource/9ikf-qahj.json",context=ssl.create_default_context(cafile=certifi.where())) as url:
    data = json.loads(url.read().decode())
data = pd.json_normalize(data)
data.to_csv(sourceDir + 'ACT-conservation-' + monthStr + '.csv',  encoding="UTF-8", index=False)
data

,category,scientificname,vernacularname,taxonrank,taxonomicstatus,kingdom,phylum,class,order,family,sourcestatus,status,authority,ngunnawal,comments,taxonremarks
0,Bird,Anthochaera phrygia,Regent Honeyeater,species,accepted,Animalia,Chordata,Aves,Passeriformes,Meliphagidae,Critically Endangered,Critically Endangered,Nature Conservation Act 2014 (ACT),NaN,NaN,NaN
1,Reptile,Aprasia parapulchella,Pink-tailed Worm-lizard,species,accepted,Animalia,Chordata,Reptilia,Squamata,Pygopodidae,Vulnerable,Vulnerable,Nature Conservation Act 2014 (ACT),Banburung,NaN,NaN
2,Mammal,Bettongia gaimardi,Eastern Bettong,species,accepted,Animalia,Chordata,Mammalia,Diprotodontia,Potoroidae,Regionally Conservation Dependent,NaN,Nature Conservation Act 2014 (ACT),Ngaluda,Regionally Conservation Dependent,NaN
3,Fish,Bidyanus bidyanus,Silver Perch,species,accepted,Animalia,Chordata,Actinopterygii,Perciformes,Terapontidae,Endangered,Endangered,Nature Conservation Act 2014 (ACT),Dhingur,NaN,NaN
4,Plant,Bossiaea grayi,Murrumbidgee Bossiaea,species,accepted,Plantae,Charophyta,Equisetopsida,Fabales,Fabaceae,Endangered,Endangered,Nature Conservation Act 2014 (ACT),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Plant,Swainsona recta,Small Purple Pea,species,accepted,Plantae,Charophyta,Equisetopsida,Fabales,Fabaceae,Endangered,Endangered,Nature Conservation Act 2014 (ACT),NaN,NaN,NaN
61,Invertebrate,Synemon plana,Golden Sun Moth,species,accepted,Animalia,Arthropoda,Insecta,Lepidoptera,Castniidae,Vulnerable,Vulnerable,Nature Conservation Act 2014 (ACT),NaN,NI2023-221,NaN
62,Plant,Thesium australe,Austral Toadflax,species,accepted,Plantae,Charophyta,Equisetopsida,Malpighiales,Santalaceae,Vulnerable,Vulnerable,Nature Conservation Act 2014 (ACT),NaN,NaN,NaN
63,Reptile,Tympanocryptis lineata,Canberra Grassland Earless Dragon,species,accepted,Animalia,Chordata,Reptilia,Squamata,Agamidae,Critically Endangered,Critically Endangered,Nature Conservation Act 2014 (ACT),Bidjiwang,NI2023-218,NaN


## Check the status values

In [38]:
data = pd.read_csv(sourceDir + 'ACT-conservation-' + monthStr + '.csv')
data.groupby(["sourcestatus","status"],dropna=False).size().sort_values(ascending=False)

sourcestatus                       status               
Vulnerable                         Vulnerable               32
Endangered                         Endangered               23
Critically Endangered              Critically Endangered     8
Regionally Conservation Dependent  NaN                       1
NaN                                Endangered                1
dtype: int64

In [39]:
# patch - set source status for the record that doesn't have it
data['sourcestatus'] = data.apply(lambda x: x['status'] if pd.isna(x['sourcestatus']) else x['sourcestatus'], axis=1)
data.groupby(["sourcestatus","status"],dropna=False).size().sort_values(ascending=False)

sourcestatus                       status               
Vulnerable                         Vulnerable               32
Endangered                         Endangered               24
Critically Endangered              Critically Endangered     8
Regionally Conservation Dependent  NaN                       1
dtype: int64

## Map statuses

In [40]:
## Map statuses
state = 'ACT'
fname = statusDir + state + "-codes.csv"
data = lf.map_status(state, fname, data)
data = data[((data['status'].notna()))]
data['status'].unique()
data

,category,scientificname,vernacularname,taxonrank,taxonomicstatus,kingdom,phylum,class,order,family,sourceStatus,status,authority,ngunnawal,comments,taxonremarks
0,Bird,Anthochaera phrygia,Regent Honeyeater,species,accepted,Animalia,Chordata,Aves,Passeriformes,Meliphagidae,Critically Endangered,Critically Endangered,Nature Conservation Act 2014 (ACT),NaN,NaN,NaN
1,Reptile,Aprasia parapulchella,Pink-tailed Worm-lizard,species,accepted,Animalia,Chordata,Reptilia,Squamata,Pygopodidae,Vulnerable,Vulnerable,Nature Conservation Act 2014 (ACT),Banburung,NaN,NaN
2,Mammal,Bettongia gaimardi,Eastern Bettong,species,accepted,Animalia,Chordata,Mammalia,Diprotodontia,Potoroidae,Regionally Conservation Dependent,Conservation Dependent,Nature Conservation Act 2014 (ACT),Ngaluda,Regionally Conservation Dependent,NaN
3,Fish,Bidyanus bidyanus,Silver Perch,species,accepted,Animalia,Chordata,Actinopterygii,Perciformes,Terapontidae,Endangered,Endangered,Nature Conservation Act 2014 (ACT),Dhingur,NaN,NaN
4,Plant,Bossiaea grayi,Murrumbidgee Bossiaea,species,accepted,Plantae,Charophyta,Equisetopsida,Fabales,Fabaceae,Endangered,Endangered,Nature Conservation Act 2014 (ACT),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Plant,Swainsona recta,Small Purple Pea,species,accepted,Plantae,Charophyta,Equisetopsida,Fabales,Fabaceae,Endangered,Endangered,Nature Conservation Act 2014 (ACT),NaN,NaN,NaN
61,Invertebrate,Synemon plana,Golden Sun Moth,species,accepted,Animalia,Arthropoda,Insecta,Lepidoptera,Castniidae,Vulnerable,Vulnerable,Nature Conservation Act 2014 (ACT),NaN,NI2023-221,NaN
62,Plant,Thesium australe,Austral Toadflax,species,accepted,Plantae,Charophyta,Equisetopsida,Malpighiales,Santalaceae,Vulnerable,Vulnerable,Nature Conservation Act 2014 (ACT),NaN,NaN,NaN
63,Reptile,Tympanocryptis lineata,Canberra Grassland Earless Dragon,species,accepted,Animalia,Chordata,Reptilia,Squamata,Agamidae,Critically Endangered,Critically Endangered,Nature Conservation Act 2014 (ACT),Bidjiwang,NI2023-218,NaN


In [41]:
# ACT only - correct typo in sourceStatus
# data['sourceStatus'] = data['sourceStatus'].replace({'Vulnernable': 'Vulnerable'})
data['status'].unique()

array(['Critically Endangered', 'Vulnerable', 'Conservation Dependent',
       'Endangered'], dtype=object)

In [42]:
consData = data.loc[:, ['scientificname', 'vernacularname', 'family', 'status', 'sourceStatus']]
consData = consData.rename(columns={'scientificname': 'scientificName', 'vernacularname': 'vernacularName'})
consData

,scientificName,vernacularName,family,status,sourceStatus
0,Anthochaera phrygia,Regent Honeyeater,Meliphagidae,Critically Endangered,Critically Endangered
1,Aprasia parapulchella,Pink-tailed Worm-lizard,Pygopodidae,Vulnerable,Vulnerable
2,Bettongia gaimardi,Eastern Bettong,Potoroidae,Conservation Dependent,Regionally Conservation Dependent
3,Bidyanus bidyanus,Silver Perch,Terapontidae,Endangered,Endangered
4,Bossiaea grayi,Murrumbidgee Bossiaea,Fabaceae,Endangered,Endangered
...,...,...,...,...,...
60,Swainsona recta,Small Purple Pea,Fabaceae,Endangered,Endangered
61,Synemon plana,Golden Sun Moth,Castniidae,Vulnerable,Vulnerable
62,Thesium australe,Austral Toadflax,Santalaceae,Vulnerable,Vulnerable
63,Tympanocryptis lineata,Canberra Grassland Earless Dragon,Agamidae,Critically Endangered,Critically Endangered


## Write conservation list to CSV

In [43]:
dr = 'dr649'
consData.to_csv(cdataDir + 'ACT-' + dr + '-conservation.csv',  encoding="UTF-8", index=False)


## Manual List check
### Instructions
1. Load the list above into the lists-test tool for this data resource
2. Unskip the below code and Run the reports below to compare to production. Send the changelog.csv to check. Correct any issues.
3. Save the production list into the historical lists directory by uncommenting the code section below.
4. Load the list from current lists into production

### Change Logs

In [44]:
ltype = "C"
changeDir = projectDir + "Monitoring/Change-logs/"

dr = "dr649"
filename = "ACT-conservation.csv"
testdr = "dr649"
proddr = "dr649"
changelist = lf.get_changelist(testdr, proddr, ltype)
changelist.to_csv(changeDir + monthStr + "-" + filename, encoding="UTF-8", index=False)
changelist

,name,scientificName,commonName,status,listUpdate,status_old
40,Melanodryas cucullata,Melanodryas (Melanodryas) cucullata,Hooded Robin,Vulnerable,added,NaN
41,Melanodryas cucullate,Melanodryas,None,Endangered,removed,NaN
64,Tympanocryptis lineata pinguicolla,Tympanocryptis pinguicolla,Grassland Earless Dragon,Critically Endangered,removed,NaN
63,Tympanocryptis pinguicolla,Tympanocryptis pinguicolla,Grassland Earless Dragon,Endangered,status change,Critically Endangered


In [45]:
# Download historical lists from Production
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + proddr + "?max=10000&includeKVP=true"
prodList = lf.download_ala_specieslist(prodListUrl)  # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/conservation/" + filename, encoding="UTF-8", index=False)
print('Finished downloading conservation historical list')


Finished downloading conservation historical list
